In [55]:
#!/usr/bin/env python3
from Crypto.Util.number import long_to_bytes, bytes_to_long
from Crypto.Random import get_random_bytes
from Crypto.Hash import SHA256
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad, unpad

import rsa

import telnetlib
import json

BLOCK_SIZE = 16

# Change this to REMOTE = False if you are running against a local instance of the server
REMOTE = False

# Remember to change the port if you are re-using this client for other challenges
PORT = 50801

if REMOTE:
    host = "aclabs.ethz.ch"
else:
    host = "localhost"

tn = telnetlib.Telnet(host, PORT)

def readline():
    return tn.read_until(b"\n")

def json_recv():
    line = readline()
    return json.loads(line.decode())

def json_send(req):
    request = json.dumps(req).encode()
    tn.write(request + b"\n")

def xor(a: bytes, b: bytes) -> bytes:
    return bytes(x ^ y for x, y in zip(a, b))   

def blockify(a):
    return [a[i : i + BLOCK_SIZE] for i in range(0, len(a), BLOCK_SIZE)]


In [56]:
json_send({
    'command': 'encrypted_flag'
})

res = json_recv()

flag_enc = bytes.fromhex(res['encypted_flag'])
flag_int = bytes_to_long(flag_enc)
N = int(res['N'], 16)
e = int(res['e'], 16)

m2 = '_{other message}'
m2_int = bytes_to_long(m2.encode())
c2 = pow(m2_int, e, N)

json_send({
    'command': 'decrypt',
    'ciphertext': flag_int + c2
})

res = json_recv()

print(res)

EOFError: telnet connection closed

In [54]:
!python3 writeup.ipynb